### ✅ 목표
- 패션 데이터를 가지고 활성화 함수, 최적화 함수를 변경시키면서 결과를 확인해보자

In [ ]:
# 필요한 라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 패션 데이터 불러오기
# 데이터 import

from tensorflow.keras.datasets import fashion_mnist

# 데이터 객체 생성

data = fashion_mnist.load_data()

In [ ]:
# 데이터 분할

X_train = data[0][0]
X_test = data[1][0]
y_train = data[0][1]
y_test = data[1][1]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# 데이터 확인

X_train
# 하나의 2차원 배열 -> 이미지 1장(x / y) -> 2차원 배열
# 이미지의 갯수까지 포함하면 1차원이 추가 되어서 3차원 배열

In [ ]:
# 정답 데이터 확인하기

y_train
# 1차원의 배열 / 숫자로 값이 이뤄졌다

In [ ]:
# 정답 고유 값을 살펴보자 - 10개의 정답(다중 분류)

np.unique(y_train)

- 📌 데이터 정리
    - 0 : 티셔츠
    - 1 : 트라우저(바지)
    - 2 : 단추가 없는 스웨터
    - 3 : 드레스
    - 4 : 코트
    - 5 : 샌들
    - 6 : 셔츠
    - 7 : 스니커즈
    - 8 : 가방
    - 9 : 앵클 부츠

In [ ]:
# 이미지 출력해 보기

plt.imshow(X_train[1], cmap='gray')

In [ ]:
plt.imshow(X_train[10000], cmap='gray')
print(y_train[10000])

In [ ]:
# 반복문을 통해서 여러 이미지를 뽑아 내보자

for i in range(1, 50, 2) :
    plt.imshow(X_train[i], cmap='gray')
    plt.show()

### ✅ 이미지 전처리
- MLP(다층 퍼셉트론)으로 신경망을 구현할 경우, 이미지를 1차원으로 펴서 넣어줘야 한다.
- 1차원으로 펴진 이미지의 픽셀은 각각 뉴런에 들어가서 학습이 진행된다.

In [ ]:
# 이미지를 1차원으로 바꿔주는 방법 - 2가지
# 1-1. reshape : 함수를 이용해서 직접 형태를 바꿔주는 방법
# 1-2. Flatten : 신경망에 층을 추가해서 1차원으로 변형 시켜주는 방법

In [ ]:
# 1-1 reshape : 손으로 직접 데이터를 펴주는 방법(실습만 해보자!)

X_train_reshape = X_train.reshape(60000, -1) # -1 -> 784, 28*28, 28**2
X_test_reshape = X_test.reshape(10000, -1)

X_train_reshape.shape, X_test_reshape.shape

In [ ]:
# 정답 데이터 전처리
# 정답 데이터 종류 : 10개 (다중 분류)

# 다중 분류를 진행 할 때 정답 데이터 -> 원핫 인코딩

# 원핫 인코딩 진행방법
# 1. pd.get_dummies() -> pandas
# 2. to_categorical() -> tensorflow

# to_categorical 사용해보기

In [ ]:
# to_categorical import

from tensorflow.keras.utils import to_categorical

In [ ]:
# 정답 데이터 원핫 인코딩 진행

y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

### ✅ 활성화 함수 / 최적화 함수 별로 성능 비교
1. 중간층 활성화 함수 : sigmoid / 최적화함수 : SGD

2. 중간층 활성화 함수 : sigmoid / 최적화 함수 : Adam

3. 중간층 활성화 함수 : relu / 최적화 함수 : Adam

4. 중간층 활성화 함수 : tanh / 최적화 함수 : SGD

In [ ]:
# 신경망을 구현하기 위한 재료 import

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# 최적화 함수 하이퍼 파라미터 변경을 위해 optimizer import

from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
# 모델 생성 함수 정의
# 활성화 함수 / 최적화 함수는 매개변수 값으로 연결
# acti = 활성화 함수, opti = 최적화 함수

def create_model(acti, opti) :
    model = Sequential()

    # 입력층 구성 - Flatten(데이터를 1차원으로 펴줌)
    model.add(Flatten(input_shape=(28, 28)))

    # MLP층 쌓아주기
    model.add(Dense(units = 200, activation=acti))
    model.add(Dense(units = 100, activation=acti))
    model.add(Dense(units = 50, activation=acti))

    # 출력층 쌓기
    model.add(Dense(units = 10, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# fit 함수 정의
# 모델명만 넣어주면 학습을 진행하는 코드를 작성

def fit_model(estimator) :
    h = estimator.fit(X_train, y_train_oh, validation_split=0.2, batch_size = 64, epochs = 30)

    return h

In [ ]:
# 1. sigmoid + SGD

model1 = create_model(acti='sigmoid', opti=SGD(learning_rate=0.01))

h1 = fit_model(model1)

In [ ]:
# 2. sigmoid + Adam

model2 = create_model(acti='sigmoid', opti=Adam(learning_rate=0.001))

h2 = fit_model(model2)

In [ ]:
# 3. relu + Adam

model3 = create_model(acti='relu', opti=Adam(learning_rate=0.001))

h3 = fit_model(model3)

In [ ]:
# 4. tanh + SGD

model4 = create_model(acti='tanh', opti=SGD(learning_rate=0.01))

h4 = fit_model(model4)

In [ ]:
# 4종류의 모델 학습 시각화
plt.figure(figsize = (15,5))

# model1
plt.plot(h1.history['accuracy'], label = 'sigmoid+SGD')

# model2
plt.plot(h2.history['accuracy'], label = 'sigmoid+Adam')

# model3
plt.plot(h3.history['accuracy'], label = 'relu+Adam')

# model4
plt.plot(h4.history['accuracy'], label = 'tanh+SGD')

plt.legend()

plt.show()